<a href="https://colab.research.google.com/github/mahtabhossain/Indoor-Localisation/blob/master/ProbabilityComputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
## suppose, there are n APs
## there names are ordered here using 1, 2, ...., n
## this is quite an important part to remember: identifier indices for each AP
n = 10  # number of total APs
lst = [x for x in range(n)]

## for each AP, build the dictionary
## histogram is kept as dictionary, e.g., if AP1=-60 is seen 4 times
## and AP1=-90 is seen 8 times, the following is the resultant dictionary for histogram
## do not consider where the values are 100: it means the AP is not seen
AP1 = {-60:4, -90:8}    # AP1 : apriori values
sumAP1 = sum(AP1.values()) # keeping track the total number of times AP1 is seen

## Also required dictionary for each AP when it is seen together with AP1
## AP2 values histogram when it is seen together with AP1
AP21 = {-60:2, -70:1} # AP2 given AP1: posteriori values
sumAP21 = sum(AP21.values()) # keeping track the total number of times AP1 & AP2 seen together

## resultant data structure
posteriori = dict() # stores all the above calculated histograms....
posteriori[0] = dict()  # stores the values where AP1 is seen

# first the apriori always as [x][x] index
# for AP1: it is [0][0], for AP2, it will be [1][1], etc.
posteriori[0][0] = [sumAP1, AP1]    # keeping as list: not as tuple - just to be safe if they need to be changed later
posteriori[0][1] = [sumAP21, AP21] 
# ..... repeat for all the other APs where it can be seen together with AP1

## this process has to be repeated for all APs
## after which P(APx=-60) and P(APx=-60|APy) can be computed from posteriori variable
print(posteriori)

{0: {0: [12, {-60: 4, -90: 8}], 1: [3, {-60: 2, -70: 1}]}}


In [7]:
## suppose, there are L locations
## for each AP, build the dictionary for P(Location_1|APx), P(Location_2|APx)......P(Location_L|APx)
AP1 = {'Location_1':2, 'Location_2':10}
sumAP1 = sum(AP1.values()) # keeping track the total number of times AP1 across all rooms

## resultant data structure for locations....
location = dict() # stores all the above calculated histograms....
location[0] = [sumAP1, AP1]    # keeping as list: not as tuple - just to be safe if they need to be changed later
# ..... repeat for all the locations where it can be seen together with AP1

## this process has to be repeated for all APs
## after which P(Room1|APx) can be computed from location variable
print(location)

## it might be even concatenated with the posteriori variable above
## may be keep it separate for simplicity

{0: [12, {'Location_1': 2, 'Location_2': 10}]}


In [0]:
## P(APy = -60 | Room1, APx = -70) type probability needs to be computed....
## may be a function
## def findProb(location, APx-identifier, APx-value, APy-identifier, APy-value):
##    need to utilise groupby, etc.
##    return the probability given the five arguments